In [1]:
import pandas as pd
import re
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Blast.Applications import NcbiblastpCommandline
import os

/home/skylite/anaconda3/lib/python3.13/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


In [2]:
###This script requires the AA sequences of desired bacteria. Use my "KEGG_ID_to_AA_seq" code for guidance to proceed.
###This script also requires a local BLAST database to be installed, this is due to NCBI Blast having a request limit from one IP.
###I recommend swissprot, at least to start out with. If you have any questions or issues email me at dtillery@sdsu.edu.

#Loads CSV. Replace the file name in quotations, make sure the script and the file are in the same folder.
df = pd.read_csv("AA_MethyOrg_Results.csv") ##Put your file name in, it has to be in .csv format.
db = "/home/skylite/blastdb/swissprot" ##this calls to where I have swissprot saved at. It will be a different directory for you wherever you install yours. Change this as well.

In [3]:
##THis puts our FASTA and BLAST requests files somewhere because there will most likely be a lot generated. Change folder names as you see fit.
fasta_folder = "fasta_queries"
os.makedirs(fasta_folder, exist_ok=True)
blast_folder = "blast_results"
os.makedirs(blast_folder, exist_ok=True)

In [ ]:
#This function will run BLAST locally through SwissProt and parse top hits, this will also save any FASTA matching files to a new folder as well as BLAST logs.
#I would recommend keeping the BLAST logs, at least temporarily, in case you need to manually look through them and see if you want to refine your search"
def run_local_blast(sequence, idx):
    #Save FASTA to fasta_folder
    fasta_path = os.path.join(fasta_folder, f"query_{idx}.fasta")
    with open(fasta_path, "w") as f:
        f.write(f">query_{idx}\n{sequence}\n")

    #Save BLAST result to blast_folder
    output_path = os.path.join(blast_folder, f"blast_result_{idx}.xml")
    blast_cline = NcbiblastpCommandline(
        query=fasta_path,
        db=db,
        evalue=0.001,
        outfmt=5,
        out=output_path
    )
    blast_cline()

    with open(output_path) as result_handle:
        blast_record = NCBIXML.read(result_handle)

    return parse_blast_hits(blast_record)

In [ ]:
#This function searches for our organism. It isn't case sensitive and I have it searching for methylobacterium. Replace the name to whatever you want.
#I recommend changing anything that says methylobacterium to how you see fit to avoid confusion.
def parse_blast_hits(blast_record):
    methylobacterium_hits = []

    for alignment in blast_record.alignments:
        hit_def = alignment.hit_def #hit_def is what it's called in BLAST. This is the gene name it'll look something like this: MMB17_RS12660 adenosylhomocysteinase [Methylobacterium extorquens]
        acc = alignment.hit_id #hit_id is an accension id, this is a unique code to the BLAST run that makes it easier to search up. You don't need to keep this.

        # Check if 'Methylobacterium' appears (case-insensitive)
        if re.search(r'methylobacterium', hit_def, re.IGNORECASE):
            #Extract organism name from brackets, if you look in the BLAST logs, you'll see it puts organism names in brackets.
            match = re.search(r'\[(.*?)\]$', hit_def)
            organism = match.group(1) if match else "Unknown_Organism"

            methylobacterium_hits.append(f"{acc} [{organism}]")

    return methylobacterium_hits if methylobacterium_hits else ["No Methylobacterium match"]

In [5]:
##Process all AA sequences, this portions ties in everything and performs BLAST on every AA sequence we found using swissprot local database##
gpr_list = []

for idx, sequence in enumerate(df['UniProt_AA_Sequences']):
    print(f"🔍 BLASTing row {idx}...")

    if pd.isna(sequence) or "No sequence" in str(sequence):
        print(f"⚠️ Skipping row {idx} — no sequence available.")
        gpr_list.append("No sequence")
        continue

    try:
        matches = run_local_blast(str(sequence), idx)

        if matches == ["No Methylobacterium match"]:
            print(f"❌ No Methylobacterium match found in row {idx}.")
            gpr_list.append("No valid GPR")
        else:
            result_str = " & ".join(matches)
            print(f"✅ Match(es) found: {result_str}")
            gpr_list.append(result_str)

    except Exception as e:
        print(f"🚨 Error in row {idx}: {e}")
        gpr_list.append("Error")

🔍 BLASTing row 0...
❌ No Methylobacterium match found in row 0.
🔍 BLASTing row 1...
❌ No Methylobacterium match found in row 1.
🔍 BLASTing row 2...
❌ No Methylobacterium match found in row 2.
🔍 BLASTing row 3...
❌ No Methylobacterium match found in row 3.
🔍 BLASTing row 4...
❌ No Methylobacterium match found in row 4.
🔍 BLASTing row 5...
✅ Match(es) found: sp|B0UI66.1| [Methylobacterium sp. 4-46] & sp|B8IUT2.1| [Methylobacterium nodulans ORS 2060]
🔍 BLASTing row 6...
⚠️ Skipping row 6 — no sequence available.
🔍 BLASTing row 7...
❌ No Methylobacterium match found in row 7.
🔍 BLASTing row 8...
❌ No Methylobacterium match found in row 8.
🔍 BLASTing row 9...
❌ No Methylobacterium match found in row 9.
🔍 BLASTing row 10...
❌ No Methylobacterium match found in row 10.
🔍 BLASTing row 11...
❌ No Methylobacterium match found in row 11.
🔍 BLASTing row 12...
❌ No Methylobacterium match found in row 12.
🔍 BLASTing row 13...
❌ No Methylobacterium match found in row 13.
🔍 BLASTing row 14...
❌ No Met

In [6]:
# Add results to DataFrame and save
df['Local_BLAST_GPR'] = gpr_list
df.to_csv("SwissProt_BLAST_GPR_Complete.csv", index=False)
print("✅ Finished. Output saved to SwissProt_BLAST_GPR_COmplete.csv")

✅ Finished. Output saved to SwissProt_BLAST_GPR_COmplete.csv
